In [7]:
import os
from dotenv import load_dotenv
load_dotenv()
GITLAB_TOKEN = os.getenv("GITLAB_TOKEN")  # Or paste your token directly (not recommended for sharing)
GITLAB_GROUP_ID = 111817240  # Use numeric group ID or group name

# You can use a unique name per run to avoid collisions:
import uuid
NEW_PROJECT_NAME = f"dummy-repo-{uuid.uuid4().hex[:6]}"
GITLAB_API_URL = "https://gitlab.com/api/v4"
headers = {"PRIVATE-TOKEN": GITLAB_TOKEN}


In [9]:
# import requests

# GITLAB_API_URL = "https://gitlab.com/api/v4"
# headers = {"PRIVATE-TOKEN": GITLAB_TOKEN}

# response = requests.get(f"{GITLAB_API_URL}/groups", headers=headers)

# if response.status_code == 200:
#     for group in response.json():
#         print(f"Name: {group['full_path']} | ID: {group['id']}")
# else:
#     print(f"Error: {response.text}")


In [8]:
import requests

# Create the new project in the group
create_repo_payload = {
    "name": NEW_PROJECT_NAME,
    "namespace_id": GITLAB_GROUP_ID,  # Group where to create the project
    "initialize_with_readme": True
}

response = requests.post(
    f"{GITLAB_API_URL}/projects",
    headers=headers,
    data=create_repo_payload
)
if response.status_code == 201:
    project = response.json()
    print(f"Created new GitLab repo: {project['web_url']}")
else:
    print(f"Failed to create repo: {response.text}")
    project = None


Created new GitLab repo: https://gitlab.com/jcdemomig-group/dummy-repo-32d0c5


In [10]:
if project:
    project_id = project["id"]

    # Create a new branch
    branch_name = "feature-branch"
    create_branch_payload = {
        "branch": branch_name,
        "ref": "main"
    }
    response = requests.post(
        f"{GITLAB_API_URL}/projects/{project_id}/repository/branches",
        headers=headers,
        data=create_branch_payload
    )
    if response.status_code == 201:
        print(f"Created branch '{branch_name}'")
    elif "already exists" in response.text:
        print(f"Branch '{branch_name}' already exists")
    else:
        print(f"Failed to create branch: {response.text}")

    # Add a file to the new branch
    file_payload = {
        "branch": branch_name,
        "commit_message": "Add test.txt",
        "actions": [
            {
                "action": "create",
                "file_path": "test.txt",
                "content": "This is a test file."
            }
        ]
    }
    response = requests.post(
        f"{GITLAB_API_URL}/projects/{project_id}/repository/commits",
        headers=headers,
        json=file_payload
    )
    if response.status_code == 201:
        print("Added file 'test.txt' to feature branch.")
    else:
        print(f"Failed to add file: {response.text}")

    # Create a merge request
    mr_payload = {
        "source_branch": branch_name,
        "target_branch": "main",
        "title": "Demo Merge Request",
        "description": "Testing automated MR creation."
    }
    response = requests.post(
        f"{GITLAB_API_URL}/projects/{project_id}/merge_requests",
        headers=headers,
        data=mr_payload
    )
    if response.status_code == 201:
        mr = response.json()
        print(f"Created Merge Request: {mr['web_url']}")
    else:
        print(f"Failed to create MR: {response.text}")


Created branch 'feature-branch'
Added file 'test.txt' to feature branch.
Created Merge Request: https://gitlab.com/jcdemomig-group/dummy-repo-32d0c5/-/merge_requests/1


In [11]:
if project:
    wiki_payload = {
        "content": "# Dummy Wiki Page\n\nThis is a test wiki page created by automation.",
        "title": "DummyWiki"
    }
    response = requests.post(
        f"{GITLAB_API_URL}/projects/{project_id}/wikis",
        headers=headers,
        data=wiki_payload
    )
    if response.status_code == 201:
        print("Wiki page created.")
    else:
        print(f"Failed to create wiki page: {response.text}")

Wiki page created.


In [12]:
if project:
    issue_payload = {
        "title": "Test Issue",
        "description": "This is an automated test issue."
    }
    response = requests.post(
        f"{GITLAB_API_URL}/projects/{project_id}/issues",
        headers=headers,
        data=issue_payload
    )
    if response.status_code == 201:
        print("Test issue created.")
    else:
        print(f"Failed to create issue: {response.text}")


Test issue created.
